<a href="https://colab.research.google.com/github/Shreya9855/Assignment/blob/main/movie_recommendation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive

drive.mount('/content/gdrive', force_remount=True)

%cd "/content/gdrive/My Drive/movie"

Mounted at /content/gdrive
/content/gdrive/My Drive/movie


In [2]:
import numpy as np
import pandas as pd

In [3]:
movies_df = pd.read_csv('tmdb_5000_movies.csv')
pd.options.mode.chained_assignment = None
movies_df.head()

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",12/10/2009,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",5/19/2007,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""...",10/26/2015,880674609,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466
3,250000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",http://www.thedarkknightrises.com/,49026,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",7/16/2012,1084939099,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106
4,260000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://movies.disney.com/john-carter,49529,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",3/7/2012,284139100,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124


**Rank based recommendation**

In [4]:
def get_recommendation(movie):
  df = movies_df[['id','genres','keywords','title','popularity','vote_count','vote_average']]                                                                                           
  df.set_index("id", drop = False) #setting id as index
  from ast import literal_eval
  df['genres'] = df['genres'].str.replace(' ',"") # remove spaces on genre
  df['keywords'] = df['keywords'].str.replace(' ',"")

  col_dict_genre = df['genres'].apply(lambda x: literal_eval(x))
  col_dict_keyword = df['keywords'].apply(lambda x: literal_eval(x))
  genre_dict = {id:genre for id,genre in zip(df['id'],col_dict_genre)}
  keyword_dict = {id:keywords for id,keywords in zip(df['id'],col_dict_keyword)}

  new_dict={}
  for id, genre in genre_dict.items():
    genre_list = []
    for data in genre:
      genre_name = data.get('name')
      genre_list.append(genre_name)
    new_dict[id] = genre_list

  new_dict_key={}
  for id, keyword in keyword_dict.items():
    keyword_list = []
    for data in keyword:
      keyword_name = data.get('name')
      keyword_list.append(keyword_name)
    new_dict_key[id] = keyword_list

  df_title = df[df.title == movie] 
  movie_id = int(df_title.id)


  movie_title_genre = new_dict[movie_id]
  movie_title_keyword = new_dict_key[movie_id]

  genre_keyword = movie_title_genre + movie_title_keyword
  # merge dictionary
  genre_keyword_dict = {}
  for id in (genre_dict.keys() | keyword_dict.keys()):
    if id in genre_dict: genre_keyword_dict.setdefault(id, []).append(genre_dict[id])
    if id in keyword_dict: genre_keyword_dict.setdefault(id, []).append(keyword_dict[id])

  count_dict = {}
  for id, value in genre_keyword_dict.items():
    if id not in count_dict.keys():
      count_dict[id] = 0
    for data in value:
      for text in data:
        if text.get('name') in movie_title_genre:
          count_dict[id] += 1
  sorted_dict = dict(sorted(count_dict.items(),key= lambda x:x[1],reverse = True))

  count_id = []
  for id, value in sorted_dict.items():
    if len(count_id) <= 10:
      count_id.append(id)
  if len(count_id) > 0:
      getdata = df[df['id'].isin(count_id)]
  else:
      getdata = df

  cols = ['id','title','popularity']
  required_df = pd.DataFrame(getdata,columns = cols)
  required_df = required_df.sort_values(by=['popularity'], ascending=False)
  required_df = required_df.loc[:, ~required_df.columns.isin(['id', 'popularity'])]
  required_df = required_df.head(10).to_string(index=False)

  return required_df


In [5]:
movies = ['My Date with Drew','Shanghai Calling','Sanctuary: Quite a Conundrum','The Puffy Chair','Frozen','The Chronicles of Narnia: Prince Caspian','Star Wars','Pirates of the Caribbean: On Stranger Tides']
for movie in movies:
  print(movie)

  print(get_recommendation(movie))
  print("")

My Date with Drew
                       title
          Jackass: The Movie
               Super Size Me
  Kevin Hart: Let Me Explain
   One Direction: This Is Us
   Michael Jordan to the Max
Dave Chappelle's Block Party
               Riding Giants
Standard Operating Procedure
The Greatest Movie Ever Sold
 To Be Frank, Sinatra at 100

Shanghai Calling
                                  title
                              Star Wars
                                  Ronin
                               Æon Flux
                      Napoleon Dynamite
                             Four Rooms
                                    Hud
Midnight in the Garden of Good and Evil
                     The Quiet American
                  The Brothers McMullen
                              Childless

Sanctuary: Quite a Conundrum
                             title
   Pride and Prejudice and Zombies
               Eight Legged Freaks
Freddy's Dead: The Final Nightmare
                       Lake Placid


**cosine similarity**

In [6]:
def get_recommendation(title):
  from sklearn.feature_extraction.text import CountVectorizer
  from sklearn.metrics.pairwise import cosine_similarity

  df = movies_df[['id','genres','keywords','title','popularity','vote_count','vote_average']]                                                                                       
  df.set_index("id", drop = False) #setting id as index
  from ast import literal_eval
  df['genres'] = df['genres'].str.replace(' ',"") # remove spaces on genre
  df['genres'] = df['genres'].str.lower()
  df['keywords'] = df['keywords'].str.replace(' ',"")
  df['keywords'] = df['keywords'].str.lower()
  col_dict_genre = df['genres'].apply(lambda x: literal_eval(x))
  col_dict_keyword = df['keywords'].apply(lambda x: literal_eval(x))
  new_dict = {}
  genre_dict = {id:genre for id,genre in zip(df['id'],col_dict_genre)}
  keyword_dict = {id:keywords for id,keywords in zip(df['id'],col_dict_keyword)}
  # merge dictionary
  for id in (genre_dict.keys() | keyword_dict.keys()):
    if id in genre_dict: new_dict.setdefault(id, []).append(genre_dict[id])
    if id in keyword_dict: new_dict.setdefault(id, []).append(keyword_dict[id])


  gen_key_dict = {}
  for id, gen_key in new_dict.items():
    gen_key_list = []
    for data in gen_key:
      for item in data:
        name = item.get('name')
        gen_key_list.append(name)
      gen_key_dict[id] = str(gen_key_list)



  df1 = pd.DataFrame(gen_key_dict.items())
  df1.columns = ['id', 'key']
  newdf = df.merge(df1, how='right')


  count_vectorizer = CountVectorizer(stop_words="english",lowercase=False)
  count_matrix = count_vectorizer.fit_transform(newdf['key'])

  cosine_sim = cosine_similarity(count_matrix,count_matrix) 
  newdf = newdf.reset_index(drop = True)
  indices = pd.Series(newdf.index, index=newdf['title'])
  indices = pd.Series(movies_df.index, index=movies_df["title"]).drop_duplicates()

  idx = indices[title]

  similarity_scores = list(enumerate(cosine_sim[idx]))
  similarity_scores= sorted(similarity_scores, key=lambda x: x[1], reverse=True)
  similarity_scores= similarity_scores[1:11]


  movies_indices = [ind[0] for ind in similarity_scores]
  
  movies = movies_df["title"].iloc[movies_indices]
  return(movies)


In [7]:
movies = ['My Date with Drew','Shanghai Calling','Sanctuary: Quite a Conundrum','The Puffy Chair','Frozen','The Chronicles of Narnia: Prince Caspian','Star Wars','Pirates of the Caribbean: On Stranger Tides']
for movie in movies:
  print(movie)

  print(get_recommendation(movie))
  print("")

My Date with Drew
2230                        Willard
3372                   Split Second
3999                            Ida
4162                         Buried
4640    Love and Other Catastrophes
4802              My Date with Drew
270                     The Martian
1074     The Indian in the Cupboard
1501                       Stigmata
3390                         Hesher
Name: title, dtype: object

Shanghai Calling
4237                                 Hatchet
1455                             Radio Flyer
1697    Indiana Jones and the Temple of Doom
2676                            Urban Legend
3352                          Action Jackson
1979                     Play It to the Bone
1440                  Flight of the Intruder
2331                    Under the Tuscan Sun
2650                     All The Queen's Men
2819                            Act of Valor
Name: title, dtype: object

Sanctuary: Quite a Conundrum
122     X-Men Origins: Wolverine
3390                      Hesher
4748